# Primeiro Projeto: Analisador Léxico

O primeiro projeto requer que você implemente um analisador léxico para a linguagem uChuck, especificado no caderno [Gramática do uChuck em EBNF](https://colab.research.google.com/drive/1GiV8weG5lzVvA7z970EiE8bUMj3DAg0x?usp=sharing). Estude a especificação da gramática do uChuck com cuidado. Para concluir este primeiro projeto, você usará o [SLY](https://sly.readthedocs.io/), uma versão Python do conjunto de ferramentas [lex/yacc](http://dinosaur.compilertools.net/) com a mesma funcionalidade mas com uma interface mais amigável. Por favor, leia o conteúdo completo desta seção e complete cuidadosamente as etapas indicadas.


## Expressões Regulares

Expressões regulares são formas concisas de descrever um conjunto de strings que atendem a um determinado padrão. Por exemplo, podemos especificar a expressão regular:

```python
r'[a-zA-Z_][0-9a-zA-Z_]*'
```

para descrever identificadores válidos na linguagem uChuck. Expressões regulares são uma mini-linguagem que permite especificar as regras para construir um conjunto de strings. Essa mini-linguagem de especificação é muito semelhante entre as diferentes linguagens de programação que contêm o conceito de expressões regulares (também chamadas de RE ou REGEX). Assim, aprender a escrever expressões regulares em Python também será útil para descrever REs em outras linguagens de programação.


## Escrevendo um Analisador Léxico
O processo de “análise léxica” consiste em ler um texto de entrada e dividí-lo em um fluxo de tokens. Cada token é como uma palavra válida do dicionário. Essencialmente, o papel do analisador léxico é simplesmente certificar-se de que o texto de entrada consiste em símbolos e tokens válidos antes de qualquer processamento adicional relacionado à análise.

Cada token é definido por uma expressão regular. Assim, sua tarefa aqui é definir um conjunto de expressões regulares para a linguagem uChuck. O trabalho real de análise léxica será tratado pelo SLY. Para um melhor entendimento, estude o capítulo [“Escrevendo um Analisador Léxico”](https://sly.readthedocs.io/en/latest/sly.html#writing-a-lexer)
da documentação do SLY.

### Especificação
Seu analisador léxico deve reconhecer os símbolos e tokens da gramática do uChuck. Por exemplo, no exemplo abaixo, o nome à esquerda é o nome do token e o valor à direita é o texto correspondente:

Palavras-chave Reservadas:
```
    WHILE : 'while'
    IF    : 'if'
    ELSE  : 'else'
```

Identificadores:
```
    ID    : qualquer texto que comece com uma letra ou '_', seguido por qualquer número
            de letras, dígitos ou sublinhados, que não seja uma palavra reservada.
```

Alguns Operadores e Delimitadores:
```
    PLUS    : '+'
    MINUS   : '-'
    TIMES   : '*'
    DIVIDE  : '/'
    CHUCk   : '=>'
    SEMI    : ';'
    LPAREN  : '('
    RPAREN  : ')'
```

Literais:
```
    INT_VAL : 123
    STRING_LIT : "Ola Mundo\n"
```

Para `INT_VAL`, você deve considerar apenas números decimais.


Comentários: Para ser ignorado pelo seu analisador léxico
```
     //             Ignora o resto da linha
     /* ... */      Ignora um bloco (nenhum aninhamento é permitido)
```

Erros: seu analisador léxico deve relatar as seguintes mensagens de erro:
```
     lineno: Unterminated comment
     lineno: Unterminated string literal
```

### Esboço do Analisador Léxico

In [1]:
import sys
!pip install sly
from sly import Lexer

In [2]:
class UChuckLexer(Lexer):
    """A lexer for the uChuck language."""

    def __init__(self, error_func):
        """Create a new Lexer.
        An error function. Will be called with an error
        message, line and column as arguments, in case of
        an error during lexing.
        """
        self.error_func = error_func

    # Reserved keywords
    keywords = {
        'int': "INT",
        'float': "FLOAT",
        'if': "IF",
        'else': "ELSE",
        'while': "WHILE",
        'break': "BREAK",
        'continue': "CONTINUE",
        'true': "TRUE",
        'false': "FALSE",
    }

    # All the tokens recognized by the lexer
    tokens = tuple(keywords.values()) + (
        # Identifiers
        "ID",
        # Constants
        "FLOAT_VAL",
        "INT_VAL",
        "STRING_LIT",
        # Operators
        "PLUS",
        "MINUS",
        "TIMES",
        "DIVIDE",
        "PERCENT",
        "LE",
        "LT",
        "GE",
        "GT",
        "EQ",
        "NEQ",
        "AND",
        "OR",
        "EXCLAMATION",
        # Assignment
        "CHUCK",
        # Delimeters
        "LPAREN",
        "RPAREN",  # ( )
        "LBRACE",
        "RBRACE",  # { }
        "L_HACK",
        "R_HACK",  # <<< >>>
        "COMMA",
        "SEMI",  # , ;
    )

    # String containing ignored characters (between tokens)
    ignore = " \t"

    # Other ignored patterns
    ignore_newline = r'\n+'
    ignore_comment = r'/\*(.|\n)*?\*/|//.*'

    # Regular expression rules for tokens
    ID = r'[a-zA-Z_][a-zA-Z0-9_]*'
    FLOAT_VAL = r'\d+\.\d+|\d+\.\d*|\.\d+'
    INT_VAL = r'\d+'
    STRING_LIT = r'"([^"\\]|\\.)*"'
    
    unterm_string = r'"(\\["\\]|[^"\\])*'
    unterm_comment = r'/\*.*'

    L_HACK = r'<<<'
    R_HACK = r'>>>'

    LE = r'<='
    GE = r'>='
    EQ = r'=='
    NEQ = r'!='
    AND = r'&&'
    OR = r'\|\|'
    CHUCK = r'=>'

    LT = r'<'
    GT = r'>'
    PLUS = r'\+'
    MINUS = r'-'
    TIMES = r'\*'
    DIVIDE = r'/'
    PERCENT = r'%'
    EXCLAMATION = r'!'

    LPAREN = r'\('
    RPAREN = r'\)'
    LBRACE = r'\{'
    RBRACE = r'\}'
    COMMA = r','
    SEMI = r';'

    # Special cases
    def ID(self, t):
      t.type = self.keywords.get(t.value, "ID")
      return t

    # Define a rule so we can track line numbers
    def ignore_newline(self, t):
        self.lineno += len(t.value)

    def ignore_comment(self, t):
        self.lineno += t.value.count("\n")

    def find_column(self, token):
        """Find the column of the token in its line."""
        last_cr = self.text.rfind('\n', 0, token.index)
        return token.index - last_cr

    # Internal auxiliary methods
    def _error(self, msg, token):
        location = self._make_location(token)
        self.error_func(msg, location[0], location[1])
        self.index += 1

    def _make_location(self, token):
        return token.lineno, self.find_column(token)

    def unterm_comment(self, t):
        msg = "Unterminated comment"
        self._error(msg, t)

    def unterm_string(self, t):
        msg = "Unterminated string literal"
        self._error(msg, t)

    def error(self, t):
        msg = "Illegal character %s" % repr(t.value[0])
        self._error(msg, t)

    # Scanner (used only for test)
    def scan(self, text):
        output = ""
        for tok in self.tokenize(text):
            print(tok)
            output += str(tok) + "\n"
        return output

In [3]:
def print_error(msg, x, y):
    # use stdout to match with the output in the .out test files
    print("Lexical error: %s at %d:%d" % (msg, x, y), file=sys.stdout)

def main(args):
    lex = UChuckLexer(print_error)
    with open(args[0], 'r') if len(args) > 0 else sys.stdin as f:
        lex.scan(f.read())

## Teste
Para o desenvolvimento inicial, tente executar o analisador léxico em um arquivo de entrada de exemplo, como:

```
/* print values of factorials */
1 => int n;
1 => int value;

while( n < 10 )
{
	value * n => value;
	<<< value >>>;
	n + 1 => n;
}
```

In [5]:
%%file test.uck
/* print values of factorials */
1 => int n;
1 => int value;

while( n < 10 )
{
	value * n => value;
	<<< value >>>;
	n + 1 => n;
}

Overwriting test.uck


E o resultado será semelhante ao texto mostrado abaixo.

```
Token(type='INT_VAL', value='1', lineno=2, index=33, end=34)
Token(type='CHUCK', value='=>', lineno=2, index=35, end=37)
Token(type='INT', value='int', lineno=2, index=38, end=41)
Token(type='ID', value='n', lineno=2, index=42, end=43)
Token(type='SEMI', value=';', lineno=2, index=43, end=44)
Token(type='INT_VAL', value='1', lineno=3, index=45, end=46)
Token(type='CHUCK', value='=>', lineno=3, index=47, end=49)
Token(type='INT', value='int', lineno=3, index=50, end=53)
Token(type='ID', value='value', lineno=3, index=54, end=59)
Token(type='SEMI', value=';', lineno=3, index=59, end=60)
Token(type='WHILE', value='while', lineno=5, index=62, end=67)
Token(type='LPAREN', value='(', lineno=5, index=67, end=68)
Token(type='ID', value='n', lineno=5, index=69, end=70)
Token(type='LT', value='<', lineno=5, index=71, end=72)
Token(type='INT_VAL', value='10', lineno=5, index=73, end=75)
Token(type='RPAREN', value=')', lineno=5, index=76, end=77)
Token(type='LBRACE', value='{', lineno=6, index=78, end=79)
Token(type='ID', value='value', lineno=7, index=81, end=86)
Token(type='TIMES', value='*', lineno=7, index=87, end=88)
Token(type='ID', value='n', lineno=7, index=89, end=90)
Token(type='CHUCK', value='=>', lineno=7, index=91, end=93)
Token(type='ID', value='value', lineno=7, index=94, end=99)
Token(type='SEMI', value=';', lineno=7, index=99, end=100)
Token(type='L_HACK', value='<<<', lineno=8, index=102, end=105)
Token(type='ID', value='value', lineno=8, index=106, end=111)
Token(type='R_HACK', value='>>>', lineno=8, index=112, end=115)
Token(type='SEMI', value=';', lineno=8, index=115, end=116)
Token(type='ID', value='n', lineno=9, index=118, end=119)
Token(type='PLUS', value='+', lineno=9, index=120, end=121)
Token(type='INT_VAL', value='1', lineno=9, index=122, end=123)
Token(type='CHUCK', value='=>', lineno=9, index=124, end=126)
Token(type='ID', value='n', lineno=9, index=127, end=128)
Token(type='SEMI', value=';', lineno=9, index=128, end=129)
Token(type='RBRACE', value='}', lineno=10, index=130, end=131)
```

In [6]:
main(["test.uck"])

Token(type='INT_VAL', value='1', lineno=2, index=33, end=34)
Token(type='CHUCK', value='=>', lineno=2, index=35, end=37)
Token(type='INT', value='int', lineno=2, index=38, end=41)
Token(type='ID', value='n', lineno=2, index=42, end=43)
Token(type='SEMI', value=';', lineno=2, index=43, end=44)
Token(type='INT_VAL', value='1', lineno=3, index=45, end=46)
Token(type='CHUCK', value='=>', lineno=3, index=47, end=49)
Token(type='INT', value='int', lineno=3, index=50, end=53)
Token(type='ID', value='value', lineno=3, index=54, end=59)
Token(type='SEMI', value=';', lineno=3, index=59, end=60)
Token(type='WHILE', value='while', lineno=5, index=62, end=67)
Token(type='LPAREN', value='(', lineno=5, index=67, end=68)
Token(type='ID', value='n', lineno=5, index=69, end=70)
Token(type='LT', value='<', lineno=5, index=71, end=72)
Token(type='INT_VAL', value='10', lineno=5, index=73, end=75)
Token(type='RPAREN', value=')', lineno=5, index=76, end=77)
Token(type='LBRACE', value='{', lineno=6, index=78,

Estude cuidadosamente a saída do analisador léxico e certifique-se de que ela faz sentido. Quando estiver razoavelmente satisfeito com a saída, tente executar alguns dos testes mais complicados projetados para testar vários cenários atípicos, fora do padrão esperado. No [AVA](https://ava2.ead.ufscar.br/mod/quiz/view.php?id=510234) há um grande conjunto de testes para verificar seu código: confira-os para ver mais exemplos.

Aqui está outro exemplo:


```
// Printing and math expressions involving ints and floats
<<< 3 + 4 * -5 >>>;
<<< 3.4 - 5.6 / -7.8 >>>;

// Characters
<<< "x" >>>;
<<< "\n" >>>;

// Constants
3.14159 => float pi;
2.0 * pi => float tau;

// Variables
float r;
float a;

// Assignment and lookup of variables
2.0 => r;
pi*r*r => a;

// Relations, booleans, and boolean expressions
true => int c;
a < 100.0 => c;
(a > 0.0) && (a < 10.0) => int d;
<<< d >>>;

// Conditionals
if( a > 0.0 )
   <<<  a >>>;
else
   <<< -a >>>;

// Loops
0 => int n;
while( n < 10 )
{
    <<< n >>>;
    n + 1 => n;
}

// Loop control flow (break/continue)
0 => n;
while( true )
{
    n + 1 => n;
    if( n > 10 )
         break;
    if( n == 5 )
         continue;
    <<< n >>>;
}
```

In [7]:
%%file test.uck
// Printing and math expressions involving ints and floats
<<< 3 + 4 * -5 >>>;
<<< 3.4 - 5.6 / -7.8 >>>;

// Characters
<<< "x" >>>;
<<< "\n" >>>;

// Constants
3.14159 => float pi;
2.0 * pi => float tau;

// Variables
float r;
float a;

// Assignment and lookup of variables
2.0 => r;
pi*r*r => a;

// Relations, booleans, and boolean expressions
true => int c;
a < 100.0 => c;
(a > 0.0) && (a < 10.0) => int d;
<<< d >>>;

// Conditionals
if( a > 0.0 )
   <<<  a >>>;
else
   <<< -a >>>;

// Loops
0 => int n;
while( n < 10 )
{
    <<< n >>>;
    n + 1 => n;
}

// Loop control flow (break/continue)
0 => n;
while( true )
{
    n + 1 => n;
    if( n > 10 )
         break;
    if( n == 5 )
         continue;
    <<< n >>>;
}

Overwriting test.uck


```
Token(type='L_HACK', value='<<<', lineno=2, index=59, end=62)
Token(type='INT_VAL', value='3', lineno=2, index=63, end=64)
Token(type='PLUS', value='+', lineno=2, index=65, end=66)
Token(type='INT_VAL', value='4', lineno=2, index=67, end=68)
Token(type='TIMES', value='*', lineno=2, index=69, end=70)
Token(type='MINUS', value='-', lineno=2, index=71, end=72)
Token(type='INT_VAL', value='5', lineno=2, index=72, end=73)
Token(type='R_HACK', value='>>>', lineno=2, index=74, end=77)
Token(type='SEMI', value=';', lineno=2, index=77, end=78)
Token(type='L_HACK', value='<<<', lineno=3, index=79, end=82)
Token(type='FLOAT_VAL', value='3.4', lineno=3, index=83, end=86)
Token(type='MINUS', value='-', lineno=3, index=87, end=88)
Token(type='FLOAT_VAL', value='5.6', lineno=3, index=89, end=92)
Token(type='DIVIDE', value='/', lineno=3, index=93, end=94)
Token(type='MINUS', value='-', lineno=3, index=95, end=96)
Token(type='FLOAT_VAL', value='7.8', lineno=3, index=96, end=99)
Token(type='R_HACK', value='>>>', lineno=3, index=100, end=103)
Token(type='SEMI', value=';', lineno=3, index=103, end=104)
Token(type='L_HACK', value='<<<', lineno=6, index=120, end=123)
Token(type='STRING_LIT', value='"x"', lineno=6, index=124, end=127)
Token(type='R_HACK', value='>>>', lineno=6, index=128, end=131)
Token(type='SEMI', value=';', lineno=6, index=131, end=132)
Token(type='L_HACK', value='<<<', lineno=7, index=133, end=136)
Token(type='STRING_LIT', value='"\\n"', lineno=7, index=137, end=141)
Token(type='R_HACK', value='>>>', lineno=7, index=142, end=145)
Token(type='SEMI', value=';', lineno=7, index=145, end=146)
Token(type='FLOAT_VAL', value='3.14159', lineno=10, index=161, end=168)
Token(type='CHUCK', value='=>', lineno=10, index=169, end=171)
Token(type='FLOAT', value='float', lineno=10, index=172, end=177)
Token(type='ID', value='pi', lineno=10, index=178, end=180)
Token(type='SEMI', value=';', lineno=10, index=180, end=181)
Token(type='FLOAT_VAL', value='2.0', lineno=11, index=182, end=185)
Token(type='TIMES', value='*', lineno=11, index=186, end=187)
Token(type='ID', value='pi', lineno=11, index=188, end=190)
Token(type='CHUCK', value='=>', lineno=11, index=191, end=193)
Token(type='FLOAT', value='float', lineno=11, index=194, end=199)
Token(type='ID', value='tau', lineno=11, index=200, end=203)
Token(type='SEMI', value=';', lineno=11, index=203, end=204)
Token(type='FLOAT', value='float', lineno=14, index=219, end=224)
Token(type='ID', value='r', lineno=14, index=225, end=226)
Token(type='SEMI', value=';', lineno=14, index=226, end=227)
Token(type='FLOAT', value='float', lineno=15, index=228, end=233)
Token(type='ID', value='a', lineno=15, index=234, end=235)
Token(type='SEMI', value=';', lineno=15, index=235, end=236)
Token(type='FLOAT_VAL', value='2.0', lineno=18, index=276, end=279)
Token(type='CHUCK', value='=>', lineno=18, index=280, end=282)
Token(type='ID', value='r', lineno=18, index=283, end=284)
Token(type='SEMI', value=';', lineno=18, index=284, end=285)
Token(type='ID', value='pi', lineno=19, index=286, end=288)
Token(type='TIMES', value='*', lineno=19, index=288, end=289)
Token(type='ID', value='r', lineno=19, index=289, end=290)
Token(type='TIMES', value='*', lineno=19, index=290, end=291)
Token(type='ID', value='r', lineno=19, index=291, end=292)
Token(type='CHUCK', value='=>', lineno=19, index=293, end=295)
Token(type='ID', value='a', lineno=19, index=296, end=297)
Token(type='SEMI', value=';', lineno=19, index=297, end=298)
Token(type='TRUE', value='true', lineno=22, index=348, end=352)
Token(type='CHUCK', value='=>', lineno=22, index=353, end=355)
Token(type='INT', value='int', lineno=22, index=356, end=359)
Token(type='ID', value='c', lineno=22, index=360, end=361)
Token(type='SEMI', value=';', lineno=22, index=361, end=362)
Token(type='ID', value='a', lineno=23, index=363, end=364)
Token(type='LT', value='<', lineno=23, index=365, end=366)
Token(type='FLOAT_VAL', value='100.0', lineno=23, index=367, end=372)
Token(type='CHUCK', value='=>', lineno=23, index=373, end=375)
Token(type='ID', value='c', lineno=23, index=376, end=377)
Token(type='SEMI', value=';', lineno=23, index=377, end=378)
Token(type='LPAREN', value='(', lineno=24, index=379, end=380)
Token(type='ID', value='a', lineno=24, index=380, end=381)
Token(type='GT', value='>', lineno=24, index=382, end=383)
Token(type='FLOAT_VAL', value='0.0', lineno=24, index=384, end=387)
Token(type='RPAREN', value=')', lineno=24, index=387, end=388)
Token(type='AND', value='&&', lineno=24, index=389, end=391)
Token(type='LPAREN', value='(', lineno=24, index=392, end=393)
Token(type='ID', value='a', lineno=24, index=393, end=394)
Token(type='LT', value='<', lineno=24, index=395, end=396)
Token(type='FLOAT_VAL', value='10.0', lineno=24, index=397, end=401)
Token(type='RPAREN', value=')', lineno=24, index=401, end=402)
Token(type='CHUCK', value='=>', lineno=24, index=403, end=405)
Token(type='INT', value='int', lineno=24, index=406, end=409)
Token(type='ID', value='d', lineno=24, index=410, end=411)
Token(type='SEMI', value=';', lineno=24, index=411, end=412)
Token(type='L_HACK', value='<<<', lineno=25, index=413, end=416)
Token(type='ID', value='d', lineno=25, index=417, end=418)
Token(type='R_HACK', value='>>>', lineno=25, index=419, end=422)
Token(type='SEMI', value=';', lineno=25, index=422, end=423)
Token(type='IF', value='if', lineno=28, index=441, end=443)
Token(type='LPAREN', value='(', lineno=28, index=443, end=444)
Token(type='ID', value='a', lineno=28, index=445, end=446)
Token(type='GT', value='>', lineno=28, index=447, end=448)
Token(type='FLOAT_VAL', value='0.0', lineno=28, index=449, end=452)
Token(type='RPAREN', value=')', lineno=28, index=453, end=454)
Token(type='L_HACK', value='<<<', lineno=29, index=458, end=461)
Token(type='ID', value='a', lineno=29, index=463, end=464)
Token(type='R_HACK', value='>>>', lineno=29, index=465, end=468)
Token(type='SEMI', value=';', lineno=29, index=468, end=469)
Token(type='ELSE', value='else', lineno=30, index=470, end=474)
Token(type='L_HACK', value='<<<', lineno=31, index=478, end=481)
Token(type='MINUS', value='-', lineno=31, index=482, end=483)
Token(type='ID', value='a', lineno=31, index=483, end=484)
Token(type='R_HACK', value='>>>', lineno=31, index=485, end=488)
Token(type='SEMI', value=';', lineno=31, index=488, end=489)
Token(type='INT_VAL', value='0', lineno=34, index=500, end=501)
Token(type='CHUCK', value='=>', lineno=34, index=502, end=504)
Token(type='INT', value='int', lineno=34, index=505, end=508)
Token(type='ID', value='n', lineno=34, index=509, end=510)
Token(type='SEMI', value=';', lineno=34, index=510, end=511)
Token(type='WHILE', value='while', lineno=35, index=512, end=517)
Token(type='LPAREN', value='(', lineno=35, index=517, end=518)
Token(type='ID', value='n', lineno=35, index=519, end=520)
Token(type='LT', value='<', lineno=35, index=521, end=522)
Token(type='INT_VAL', value='10', lineno=35, index=523, end=525)
Token(type='RPAREN', value=')', lineno=35, index=526, end=527)
Token(type='LBRACE', value='{', lineno=36, index=528, end=529)
Token(type='L_HACK', value='<<<', lineno=37, index=534, end=537)
Token(type='ID', value='n', lineno=37, index=538, end=539)
Token(type='R_HACK', value='>>>', lineno=37, index=540, end=543)
Token(type='SEMI', value=';', lineno=37, index=543, end=544)
Token(type='ID', value='n', lineno=38, index=549, end=550)
Token(type='PLUS', value='+', lineno=38, index=551, end=552)
Token(type='INT_VAL', value='1', lineno=38, index=553, end=554)
Token(type='CHUCK', value='=>', lineno=38, index=555, end=557)
Token(type='ID', value='n', lineno=38, index=558, end=559)
Token(type='SEMI', value=';', lineno=38, index=559, end=560)
Token(type='RBRACE', value='}', lineno=39, index=561, end=562)
Token(type='INT_VAL', value='0', lineno=42, index=602, end=603)
Token(type='CHUCK', value='=>', lineno=42, index=604, end=606)
Token(type='ID', value='n', lineno=42, index=607, end=608)
Token(type='SEMI', value=';', lineno=42, index=608, end=609)
Token(type='WHILE', value='while', lineno=43, index=610, end=615)
Token(type='LPAREN', value='(', lineno=43, index=615, end=616)
Token(type='TRUE', value='true', lineno=43, index=617, end=621)
Token(type='RPAREN', value=')', lineno=43, index=622, end=623)
Token(type='LBRACE', value='{', lineno=44, index=624, end=625)
Token(type='ID', value='n', lineno=45, index=630, end=631)
Token(type='PLUS', value='+', lineno=45, index=632, end=633)
Token(type='INT_VAL', value='1', lineno=45, index=634, end=635)
Token(type='CHUCK', value='=>', lineno=45, index=636, end=638)
Token(type='ID', value='n', lineno=45, index=639, end=640)
Token(type='SEMI', value=';', lineno=45, index=640, end=641)
Token(type='IF', value='if', lineno=46, index=646, end=648)
Token(type='LPAREN', value='(', lineno=46, index=648, end=649)
Token(type='ID', value='n', lineno=46, index=650, end=651)
Token(type='GT', value='>', lineno=46, index=652, end=653)
Token(type='INT_VAL', value='10', lineno=46, index=654, end=656)
Token(type='RPAREN', value=')', lineno=46, index=657, end=658)
Token(type='BREAK', value='break', lineno=47, index=668, end=673)
Token(type='SEMI', value=';', lineno=47, index=673, end=674)
Token(type='IF', value='if', lineno=48, index=679, end=681)
Token(type='LPAREN', value='(', lineno=48, index=681, end=682)
Token(type='ID', value='n', lineno=48, index=683, end=684)
Token(type='EQ', value='==', lineno=48, index=685, end=687)
Token(type='INT_VAL', value='5', lineno=48, index=688, end=689)
Token(type='RPAREN', value=')', lineno=48, index=690, end=691)
Token(type='CONTINUE', value='continue', lineno=49, index=701, end=709)
Token(type='SEMI', value=';', lineno=49, index=709, end=710)
Token(type='L_HACK', value='<<<', lineno=50, index=715, end=718)
Token(type='ID', value='n', lineno=50, index=719, end=720)
Token(type='R_HACK', value='>>>', lineno=50, index=721, end=724)
Token(type='SEMI', value=';', lineno=50, index=724, end=725)
Token(type='RBRACE', value='}', lineno=51, index=726, end=727)
```

In [8]:
main(["test.uck"])

Token(type='L_HACK', value='<<<', lineno=2, index=59, end=62)
Token(type='INT_VAL', value='3', lineno=2, index=63, end=64)
Token(type='PLUS', value='+', lineno=2, index=65, end=66)
Token(type='INT_VAL', value='4', lineno=2, index=67, end=68)
Token(type='TIMES', value='*', lineno=2, index=69, end=70)
Token(type='MINUS', value='-', lineno=2, index=71, end=72)
Token(type='INT_VAL', value='5', lineno=2, index=72, end=73)
Token(type='R_HACK', value='>>>', lineno=2, index=74, end=77)
Token(type='SEMI', value=';', lineno=2, index=77, end=78)
Token(type='L_HACK', value='<<<', lineno=3, index=79, end=82)
Token(type='FLOAT_VAL', value='3.4', lineno=3, index=83, end=86)
Token(type='MINUS', value='-', lineno=3, index=87, end=88)
Token(type='FLOAT_VAL', value='5.6', lineno=3, index=89, end=92)
Token(type='DIVIDE', value='/', lineno=3, index=93, end=94)
Token(type='MINUS', value='-', lineno=3, index=95, end=96)
Token(type='FLOAT_VAL', value='7.8', lineno=3, index=96, end=99)
Token(type='R_HACK', val

## Anexo
A lista completa de tokens usados na linguagem uChuck é:

```python
    # Reserved keywords
    keywords = {
        'int': "INT",
        'float': "FLOAT",
        'if': "IF",
        'else': "ELSE",
        'while': "WHILE",
        'break': "BREAK",
        'continue': "CONTINUE",
        'true': "TRUE",
        'false': "FALSE",
    }

    # All the tokens recognized by the lexer
    tokens = tuple(keywords.values()) + (
        # Identifiers
        "ID",
        # Constants
        "FLOAT_VAL",
        "INT_VAL",
        "STRING_LIT",
        # Operators
        "PLUS",
        "MINUS",
        "TIMES",
        "DIVIDE",
        "PERCENT",
        "LE",
        "LT",
        "GE",
        "GT",
        "EQ",
        "NEQ",
        "AND",
        "OR",
        "EXCLAMATION",
        # Assignment
        "CHUCK",
        # Delimeters
        "LPAREN",
        "RPAREN",  # ( )
        "LBRACE",
        "RBRACE",  # { }
        "L_HACK",
        "R_HACK",  # <<< >>>
        "COMMA",
        "SEMI",  # , ;
    )
```